<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [48]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Apr 14 16:13:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              42W / 300W |  15126MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Use the following cells to download the data.

In [49]:
from google.colab import drive
import requests
import zipfile

# Mount Google Drive
drive.mount('/content/drive')

# URL of the ZIP file
zip_url = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"

# Destination directory in Google Drive
destination_dir = "/content/drive/My Drive/"

# Download the ZIP file using requests
response = requests.get(zip_url)
if response.status_code == 200:
    # Save the downloaded ZIP file to Google Drive
    with open(destination_dir + "concrete_data_week4.zip", 'wb') as f:
        f.write(response.content)
    print("File downloaded successfully.")

    # Path to the downloaded ZIP file
    zip_file_path = destination_dir + "concrete_data_week4.zip"

    # Destination directory for extracting the ZIP file
    extract_dir = "/content/concrete_data_week4/"

    # Extract the contents of the ZIP file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    print("ZIP file extracted successfully.")
else:
    print("Error downloading the file. Status code:", response.status_code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File downloaded successfully.
ZIP file extracted successfully.


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

In [3]:
import os

# Destination directory where the ZIP file was extracted
extract_dir = "/content/concrete_data_week4/"

# List the contents of the 'concrete_data_week4' directory
subfolder_contents = os.listdir(extract_dir + "concrete_data_week4/")

# Print the contents of the 'concrete_data_week4' directory
print("Contents of the 'concrete_data_week4' directory:")
for item in subfolder_contents:
    print(item)


Contents of the 'concrete_data_week4' directory:
train
test
valid
.DS_Store


<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [51]:
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

In [52]:
# Import preprocess_input function from keras.applications.vgg16
from keras.applications.vgg16 import preprocess_input

In [53]:
# Use a batch size of 100 images for both training and validation
batch_size = 100

In [54]:
# Construct an ImageDataGenerator for the training set with augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Preprocess to match VGG16 requirements
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [55]:
# Construct an ImageDataGenerator for the validation set
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [56]:
# Create a sequential model and add VGG16 as the base model, without its top layers
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential([vgg_model])

In [57]:
# Flatten the output of the VGG16 to connect it to dense layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))  # Intermediate dense layer with relu activation
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

In [58]:
# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [59]:
# Fit the model on the augmented data using the ImageDataGenerators
train_generator = train_datagen.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/train',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 30001 images belonging to 2 classes.


In [60]:
val_generator = val_datagen.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/valid',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

Found 9501 images belonging to 2 classes.


In [61]:
# Use the fit method (fit_generator is deprecated in newer versions of TensorFlow)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=5  # Adjust number of epochs as needed
)

Epoch 1/5
300/300 [==============================] - 449s 1s/step - loss: 0.8724 - accuracy: 0.9561 - val_loss: 0.0449 - val_accuracy: 0.9798
Epoch 2/5
300/300 [==============================] - 443s 1s/step - loss: 0.0178 - accuracy: 0.9946 - val_loss: 0.0172 - val_accuracy: 0.9938
Epoch 3/5
300/300 [==============================] - 455s 2s/step - loss: 0.0130 - accuracy: 0.9963 - val_loss: 0.0098 - val_accuracy: 0.9966
Epoch 4/5
300/300 [==============================] - 447s 1s/step - loss: 0.0142 - accuracy: 0.9960 - val_loss: 0.0084 - val_accuracy: 0.9965
Epoch 5/5
300/300 [==============================] - 456s 2s/step - loss: 0.0090 - accuracy: 0.9976 - val_loss: 0.0187 - val_accuracy: 0.9944


In [62]:
model.save('model')  # Saves the model in HDF5 format


## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [63]:
from keras.preprocessing.image import ImageDataGenerator

In [64]:
from keras.models import load_model

# load resnet model
resnet_path = '/content/classifier_resnet_model.h5'
resnet = load_model(resnet_path)


In [65]:
print(resnet.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
None


In [66]:
from keras.layers import Dense
from keras.models import Model

# Remove the last layer
resnet.layers.pop()

# Add a new Dense layer with 1 output unit and sigmoid activation
new_output = Dense(1, activation='sigmoid')(resnet.layers[-1].output)
resnet = Model(inputs=resnet.input, outputs=new_output)

# Recompile the model
resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Check the updated model summary
print(resnet.summary())


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50_input (InputLayer  [(None, None, None, 3)]   0         
 )                                                               
                                                                 
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 23591813 (90.00 MB)
Trainable params: 4101 (16.02 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
None


In [67]:
resnet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [68]:
# Directory of the test dataset
test_dir = '/content/concrete_data_week4/concrete_data_week4/test'

In [69]:
# Create an instance of ImageDataGenerator with the necessary configurations
# Normalize the images to match the preprocessing used in model training
test_datagen = ImageDataGenerator(rescale=1./255)

In [70]:
# Create a generator that loads the test images from the specified directory
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Standard size for VGG16 and ResNet models
    batch_size=20,           # Batch size can be adjusted based on your system's capabilities
    class_mode='binary',     # For binary classification problems
    shuffle=False)           # Shuffle is set to False for evaluation

Found 500 images belonging to 2 classes.


In [71]:
# Evaluate performance
vgg16_performance = model.evaluate(test_generator, steps=len(test_generator))
resnet_performance = resnet.evaluate(test_generator, steps=len(test_generator))

25/25 [==============================] - 3s 37ms/step - loss: 0.7231 - accuracy: 0.5000


In [72]:
# Print the performance
print("Performance of the VGG16 model:")
print("Loss: {:.4f}, Accuracy: {:.4f}".format(vgg16_performance[0], vgg16_performance[1]))

print("Performance of the ResNet model:")
print("Loss: {:.4f}, Accuracy: {:.4f}".format(resnet_performance[0], resnet_performance[1]))


Performance of the VGG16 model:
Loss: 5.1038, Accuracy: 0.5000
Performance of the ResNet model:
Loss: 0.7231, Accuracy: 0.5000


In [ ]:
from google.colab import files
files.download('model')


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [73]:
# predict classes

from keras.preprocessing.image import ImageDataGenerator

# Assuming you've already defined 'test_datagen' and 'test_generator' previously
# Predict the probabilities or class labels
predictions_vgg16 = model.predict(test_generator, steps=len(test_generator))
predictions_resnet = resnet.predict(test_generator, steps=len(test_generator))

# Since it's a binary classification, you may receive probabilities. To get class labels:
predicted_classes_vgg16 = (predictions_vgg16 > 0.5).astype(int)
predicted_classes_resnet = (predictions_resnet > 0.5).astype(int)


25/25 [==============================] - 2s 33ms/step


In [74]:
# Print the predictions for the first five images with 'negative' for 0 and 'positive' for 1
print("First five predictions by the VGG16 model:")
print(['positive' if x == 1 else 'negative' for x in predicted_classes_vgg16[:5].flatten()])

print("First five predictions by the ResNet model:")
print(['positive' if x == 1 else 'negative' for x in predicted_classes_resnet[:5].flatten()])


First five predictions by the VGG16 model:
['negative', 'negative', 'negative', 'negative', 'negative']
First five predictions by the ResNet model:
['positive', 'positive', 'positive', 'positive', 'positive']


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).